In [ ]:
import unittest
import numpy

# import sys
# sys.path.append('../src')

from src.basis_class import Basis

ImportError: attempted relative import with no known parent package

In [6]:
class BasisTestCase(unittest.TestCase):
    def test_get_correct_dimensions(self):
        n = 2
        basis_class = Basis(2)
        result = basis_class.dimension
        self.assertEquals(result, n)

        

In [7]:
unittest.main(argv=[''], exit=False)

E
ERROR: test_get_correct_dimensions (__main__.BasisTestCase.test_get_correct_dimensions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\waqas\AppData\Local\Temp\ipykernel_32712\4248421554.py", line 4, in test_get_correct_dimensions
    basis_class = Basis(2)
                  ^^^^^
NameError: name 'Basis' is not defined

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (errors=1)
